# How to: Chat models

Resource: https://python.langchain.com/docs/how_to/#chat-models

In [1]:
import sys

sys.path.append("../")

from custom_features.models import MODELS
from custom_features.params_setting_fct import set_api_keys, set_langsmith

set_api_keys()
set_langsmith()

## How to use chat models to call tools

Resource: https://python.langchain.com/docs/how_to/tool_calling/

In [ ]:
from langchain_core.tools import tool
from langchain_mistralai.chat_models import ChatMistralAI

In [ ]:
@tool(parse_docstring=True, response_format="content_and_artifact")
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b

@tool(parse_docstring=True, response_format="content_and_artifact")
def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b


In [ ]:
from pydantic import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

In [ ]:
# llm = ChatMistralAI(model_name=MODELS[0]["model"], max_retries=5)
llm = ChatMistralAI(model_name="mistral-large-latest", max_retries=5)


In [ ]:
llm_with_tools = llm.bind_tools([add, multiply])

In [ ]:
query = "What is 3 + 12?"

llm_with_tools.invoke(query)

In [ ]:
query = "What is 3 * 12? And, what is 11 + 49?"

called_tools = llm_with_tools.invoke(query)

print(called_tools.tool_calls)
print(called_tools.invalid_tool_calls)

In [ ]:
from langchain_core.output_parsers import PydanticToolsParser
from pydantic import BaseModel, Field

In [ ]:
class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

In [ ]:
llm_with_tools = llm.bind_tools([add, multiply])

chain = llm_with_tools | PydanticToolsParser(tools=[add, multiply])
chain.invoke(query)

In [ ]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field
from langchain_core.tools import ToolException


class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


def add_integers(a: int, b: int) -> str:
    """Add two integers."""
    return f"{a + b}"


def multiply_integers(a: int, b: int) -> str:
    """Multiply two integers."""
    return a * b
    # content = a * b
    # if content <= 20:
    #     return content
    # else:
    #     raise ToolException(f"multiplication should be superior or equal to '20'. Found '{content}'")

add_calculator = StructuredTool.from_function(func=add_integers,
                                              name="Addition",
                                              description="add integers",
                                              args_schema=CalculatorInput,
                                              return_direct=False,  # default is False
                                              handle_tool_error=True)

multiply_calculator = StructuredTool.from_function(func=multiply_integers,
                                                   name="Multiply",
                                                   description="multiply integers",
                                                   args_schema=CalculatorInput,
                                                   return_direct=False,  # default is False
                                                   response_format="content",
                                                   handle_tool_error=True)

llm_with_tools_errors_handling = llm.bind_tools([add_calculator, multiply_calculator])

In [ ]:
query_test = "What 3 * 6?"

In [ ]:
llm_with_tools_errors_handling.invoke(query_test)

In [ ]:
add_calculator.invoke({"a": 4, "b": 5})

In [ ]:
multiply_calculator.invoke({"name": "mulitply_calculator",
                            "args": {"a": 3, "b": 7},
                            "id": "123",
                            "type": "tool_call"}
                            )

In [ ]:
multiply_calculator.invoke({"a": 3, "b": 7})

## How to pass tool outputs to chat models

Resource: https://python.langchain.com/docs/how_to/tool_results_pass_to_model/

In [ ]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model_name=MODELS[0]["model"], max_retries=5)

In [ ]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


In [ ]:
tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools=tools)

In [ ]:
from langchain_core.messages import HumanMessage

In [ ]:
query = "What is 41 + 74? Also, what is 3 * 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

ai_msg

In [ ]:
ai_msg.tool_calls

In [ ]:
messages.append(ai_msg)

In [ ]:
messages

In [ ]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

In [ ]:
ai_msg = llm_with_tools.invoke(messages)
ai_msg

## How to return structured data from a model

Resource: https://python.langchain.com/docs/how_to/structured_output/

In [2]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model_name=MODELS[0]["model"], max_retries=5)
llm

ChatMistralAI(client=<httpx.Client object at 0x00000213EC56C130>, async_client=<httpx.AsyncClient object at 0x00000213EE6BFBB0>, mistral_api_key=SecretStr('**********'), endpoint='https://api.mistral.ai/v1', model='open-mistral-7b')

### Pydantic structured output

In [3]:
from typing import Optional
from pydantic import BaseModel, Field

In [4]:
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline of the joke")
    rating: Optional[int] = Field(default=None,
                                  description="How funny the joke is, from 1 to 10")
    

In [5]:
class UselessKnowledge(BaseModel):
    """Teach a useless knowledge to shine in society."""

    knowledge: str = Field(description="The useless knowledge to teach")
    rating: Optional[int] = Field(default=None,
                                  description=("How useless the knowledge is, "
                                               "from 1 (very useful) to 10 (very useless)")
    )

In [6]:
structured_llm = llm.with_structured_output(Joke)
structured_llm = llm.with_structured_output(UselessKnowledge)


In [7]:
structured_llm.invoke("Hello, I'd like to learn something")

UselessKnowledge(knowledge="Did you know that a group of flamingos is called a 'flamboyance'?", rating=None)

### TypedDict structured output

### JSON Schema structured output